<a href="https://colab.research.google.com/github/shreyus3003/Deep_Learning/blob/master/Home_work_4_Graded/VGG_16_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from torch.nn import Module
from torch import nn
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import CIFAR100
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [0]:

cfg = {
    'D' : [64, 64, 64, 128,'M', 128, 128, 'M', 256, 256, 256, 256,      'M', 512, 512, 512, 512,      'M', 512, 512, 512, 512,      'M'],
}
# def __init__(self):
      
#         super(VGG16, self).__init__()
#         self.conv1_1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
#         #nn.Conv2d()
#         self.conv1_2 = nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
#         self.conv2_1 = nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
#         self.conv2_2 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
#         self.conv3_1 = nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
#         self.conv3_2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
#         self.conv3_3 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
#         self.conv4_1 = nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
#         self.conv4_2 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
#         self.conv4_3 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
#         self.conv5_1 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
#         self.conv5_2 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
#         self.conv5_3 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
#         self._pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
#         self.sig = nn.LogSoftmax(dim=-1)
#         # self._pool = rm.MaxPool2d(filter=2, stride=2)

#     def forward(self, x):
#         c1_1 = nn.ReLU(self.conv1_1(x))
#         c1_2 = nn.ReLU(self.conv1_2(c1_1))
#         p1 = self._pool(c1_2)

#         c2_1 = nn.ReLU(self.conv2_1(p1))
#         c2_2 = nn.ReLU(self.conv2_2(c2_1))
#         p2 = self._pool(c2_2)

#         c3_1 = nn.ReLU(self.conv3_1(p2))
#         c3_2 = nn.ReLU(self.conv3_2(c3_1))
#         c3_3 = nn.ReLU(self.conv3_3(c3_2))
#         p3 = self._pool(c3_3)

#         c4_1 = nn.ReLU(self.conv4_1(p3))
#         c4_2 = nn.ReLU(self.conv4_2(c4_1))
#         c4_3 = nn.ReLU(self.conv4_3(c4_2))
#         p4 = self._pool(c4_3)

#         c5_1 = nn.ReLU(self.conv5_1(p4))
#         c5_2 = nn.ReLU(self.conv5_2(c5_1))
#         c5_3 = nn.ReLU(self.conv5_3(c5_2))
#         output = self.sig(c5_3)


class VGG16(nn.Module):

    def __init__(self, features, num_class=100):
        super().__init__()
        self.features = features

        self.classifier = nn.Sequential(
            nn.Linear(512, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_class)
        )

    def forward(self, x):
        output = self.features(x)
        output = output.view(output.size()[0], -1)
        output = self.classifier(output)
    
        return output

def make_layers(cfg, batch_norm=False):
    layers = []

    input_channel = 3
    for l in cfg:
        if l == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            continue

        layers += [nn.Conv2d(input_channel, l, kernel_size=3, padding=1)]

        if batch_norm:
            layers += [nn.BatchNorm2d(l)]
        
        layers += [nn.ReLU(inplace=True)]
        input_channel = l
    
    return nn.Sequential(*layers)

def vgg16_bn():
    return VGG(make_layers(cfg['D'], batch_norm=True))


In [3]:
data_train = CIFAR100('./data/cifar100',download=True,transform=transforms.Compose([transforms.Resize((32, 32)),transforms.ToTensor()]))
data_test = CIFAR100('./data/cifar100',train=False,download=True,transform=transforms.Compose([transforms.Resize((32, 32)),transforms.ToTensor()]))
# data_train = CIFAR100('./data/cifar100',download=True,transform=transforms.Compose([transforms.Resize((32, 32))]))
# data_test = CIFAR100('./data/cifar100',train=False,download=True,transform=transforms.Compose([transforms.Resize((32, 32))]))
data_train_loader = DataLoader(data_train, batch_size=256, shuffle=True, num_workers=8)
data_test_loader = DataLoader(data_test, batch_size=1024, num_workers=8)

# trainset = torchvision.datasets.CIFAR100(root=dataroot,
#                                          train=True,
#                                          download=True,
#                                          transform=transform_train)
# trainloader = torch.utils.data.DataLoader(
#     trainset, batch_size=batch_size_train, shuffle=True, num_workers=4)

# testset = torchvision.datasets.CIFAR100(root=dataroot,
#                                         train=False,
#                                         download=True,
#                                         transform=transform_test)
# testloader = torch.utils.data.DataLoader(
#     testset, batch_size=batch_size_test, shuffle=False, num_workers=4)



Extracting ./data/cifar100/cifar-100-python.tar.gz to ./data/cifar100
Files already downloaded and verified


In [6]:
net = VGG16(make_layers(cfg['D'], batch_norm=True))
criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(net.parameters(), lr=2e-4)
#optimizer = optim.SGD(net.parameters(), lr=2e-4, momentum=0.9, weight_decay=5e-4)
# if use_gpu:
#     vgg16.cuda() #.cuda() will move everything to the GPU side
    
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

def train(epoch):
    net.train()
    loss_list, batch_list = [], []
    for i, (images, labels) in enumerate(data_train_loader):
        optimizer.zero_grad()

        output = net(images)

        loss = criterion(output, labels)

        loss_list.append(loss.detach().cpu().item())
        batch_list.append(i+1)

        if i % 10 == 0:
          print('Train - Epoch %d, Batch: %d, Loss: %f' % (epoch, i, loss.detach().cpu().item()))
        loss.backward()
        optimizer.step()

def test():
    net.eval()
    total_correct = 0
    avg_loss = 0.0
    for i, (images, labels) in enumerate(data_test_loader):
        output = net(images)
        avg_loss += criterion(output, labels).sum()
        pred = output.detach().max(1)[1]
        total_correct += pred.eq(labels.view_as(pred)).sum()

    avg_loss /= len(data_test)
    print('Test Avg. Loss: %f, Accuracy: %f' % (avg_loss.detach().cpu().item(), float(total_correct) / len(data_test)))
    print("\n")


def train_and_test(epoch):
    train(epoch)
    test()

def main():
    for epoch in range(0, 10):
        train_and_test(epoch)


if __name__ == '__main__':
    main()

Train - Epoch 0, Batch: 0, Loss: 4.653102
Train - Epoch 0, Batch: 10, Loss: 4.616851
Train - Epoch 0, Batch: 20, Loss: 4.593169
Train - Epoch 0, Batch: 30, Loss: 4.565446
Train - Epoch 0, Batch: 40, Loss: 4.599750
Train - Epoch 0, Batch: 50, Loss: 4.504120
Train - Epoch 0, Batch: 60, Loss: 4.564486
Train - Epoch 0, Batch: 70, Loss: 4.513207
Train - Epoch 0, Batch: 80, Loss: 4.532394
Train - Epoch 0, Batch: 90, Loss: 4.531451
Train - Epoch 0, Batch: 100, Loss: 4.469094
Train - Epoch 0, Batch: 110, Loss: 4.513533
Train - Epoch 0, Batch: 120, Loss: 4.494106
Train - Epoch 0, Batch: 130, Loss: 4.442388
Train - Epoch 0, Batch: 140, Loss: 4.493900
Train - Epoch 0, Batch: 150, Loss: 4.406970
Train - Epoch 0, Batch: 160, Loss: 4.454669
Train - Epoch 0, Batch: 170, Loss: 4.455272
Train - Epoch 0, Batch: 180, Loss: 4.403358
Train - Epoch 0, Batch: 190, Loss: 4.372173


RuntimeError: ignored

In [0]:
class VGG16(nn.Module):

    def __init__(self):
      
        super(VGG16, self).__init__()
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        #nn.Conv2d()
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self._pool = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.sig = nn.LogSoftmax(dim=-1)
        # self._pool = rm.MaxPool2d(filter=2, stride=2)

    def forward(self, x):
        c1_1 = nn.ReLU(self.conv1_1(x))
        c1_2 = nn.ReLU(self.conv1_2(c1_1))
        p1 = self._pool(c1_2)

        c2_1 = nn.ReLU(self.conv2_1(p1))
        c2_2 = nn.ReLU(self.conv2_2(c2_1))
        p2 = self._pool(c2_2)

        c3_1 = nn.ReLU(self.conv3_1(p2))
        c3_2 = nn.ReLU(self.conv3_2(c3_1))
        c3_3 = nn.ReLU(self.conv3_3(c3_2))
        p3 = self._pool(c3_3)

        c4_1 = nn.ReLU(self.conv4_1(p3))
        c4_2 = nn.ReLU(self.conv4_2(c4_1))
        c4_3 = nn.ReLU(self.conv4_3(c4_2))
        p4 = self._pool(c4_3)

        c5_1 = nn.ReLU(self.conv5_1(p4))
        c5_2 = nn.ReLU(self.conv5_2(c5_1))
        c5_3 = nn.ReLU(self.conv5_3(c5_2))
        output = self.sig(c5_3)
        return output